In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

# Function to extract HOG features from an image
def extract_features(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    features, _ = hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell,
                      cells_per_block=cells_per_block, visualize=True, transform_sqrt=True)
    return features

# Function to slide a window across an image
def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size[1], step_size[1]):
        for x in range(0, image.shape[1] - window_size[0], step_size[0]):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

# Load the image
image = cv2.imread("image_path.jpg")

# Define parameters
window_size = (64, 128)
step_size = (8, 8)
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

# Load the trained Linear SVM classifier
svm = LinearSVC()
svm.load("svm_model.pkl")  # Load your trained SVM model

# Initialize lists to store detections
detections = []

# Slide the window across the image and classify each window
for (x, y, window) in sliding_window(image, step_size, window_size):
    # Resize the window to the required input size for the SVM
    window = cv2.resize(window, (64, 128))  # Adjust the size based on your model

    # Extract HOG features from the window
    features = extract_features(window, orientations, pixels_per_cell, cells_per_block)

    # Scale the features
    features = np.array(features).reshape(1, -1)
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    # Predict using the SVM classifier
    prediction = svm.predict(features)

    # If the prediction is positive, save the coordinates of the detection
    if prediction == 1:  # Modify this condition based on your class labels
        detections.append((x, y, window_size[0], window_size[1]))

# Draw bounding boxes around detected objects
for (x, y, w, h) in detections:
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the result
cv2.imshow("Object Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()